In [1]:
import torch
from det3d.torchie import Config

import pickle

config = "/workspace/centerformer/configs/nusc/nuscenes_centerformer_poolformer.py"

cfg = Config.fromfile(config)
FINDCENTER_GEN_ONNX = False
POOLFORMER_GEN_ONNX = False

In [2]:
from det3d.models import build_detector
from det3d.torchie.trainer import load_checkpoint

checkpoint_path = "/workspace/centerformer/work_dirs/nuscenes_poolformer/poolformer.pth"

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu");

Deformable Convolution not built!
Deformable Convolution not built!
Use HM Bias:  -2.19


/data/centerformer/det3d/core/bbox/geometry.py:149: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:162: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:280: NumbaDeprecationWarning: The 'nopython' keyword argument was

In [3]:
from torch import nn

class CenterFinder(nn.Module):
    def __init__(self, model):
        super(CenterFinder, self).__init__()
        self.model = model

    def forward(self, x):
        # x = self.model.neck(x)
        # preds = self.model.bbox_head(x)
        # return preds
        return self.model.neck.find_centers(x) # pos_features, out_dict_list, ct_feat 
    
centerFinder = CenterFinder(model)
centerFinder.cuda()
centerFinder.eval();

In [4]:
import pickle
# with open("/workspace/centerformer/work_dirs/partition/sample_data/x.pkl", 'wb') as handle:
#     pickle.dump(x, handle)
# with open("/workspace/centerformer/work_dirs/partition/sample_data/example.pkl", 'wb') as handle:
#     pickle.dump(example, handle)

pickle_dir = "/workspace/centerformer/work_dirs/partition/sample_data/"

with open(pickle_dir + "findcenter_input.pkl", 'rb') as handle:
    x = pickle.load(handle)
# with open(pickle_dir + "example.pkl", 'rb') as handle:
#     example = pickle.load(handle)

In [5]:
with torch.no_grad():
    ct_feat, center_pos_embedding, out_dict_list = centerFinder(x)
    
with open(pickle_dir + "findcenter_output1.pkl", 'rb') as handle:
    output1 = pickle.load(handle)
with open(pickle_dir + "findcenter_output2.pkl", 'rb') as handle:
    output2 = pickle.load(handle)
with open(pickle_dir + "findcenter_output3.pkl", 'rb') as handle:
    output3 = pickle.load(handle)
    
    
print((output1 == ct_feat).all())
print((output2 == center_pos_embedding).all())

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')


/root/anaconda3/envs/centerformer/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [6]:
import onnx

if FINDCENTER_GEN_ONNX:
    model_name = "findCenter.onnx"
    torch.onnx.export(centerFinder, x, model_name, do_constant_folding=True,
                export_params=True, opset_version=11)
    
    # !export POLYGRAPHY_AUTOINSTALL_DEPS=1
    !polygraphy surgeon sanitize findCenter.onnx --fold-constants --output findCenter_folded.onnx

    onnx.checker.check_model(onnx.load("findCenter_folded.onnx"))
    print("gen findCenter.onnx success!")
else:
    print("pass")

pass


In [7]:
del centerFinder

class NeckPoolformer(nn.Module):
    def __init__(self, model):
        super(NeckPoolformer, self).__init__()
        self.model = model

    def forward(self, ct_feat, center_pos):
        return self.model.neck.poolformer_forward(ct_feat, center_pos) # pos_features, out_dict_list, ct_feat 
    
poolformer = NeckPoolformer(model)
poolformer.cuda()
poolformer.eval();

In [8]:
with torch.no_grad():
    output = poolformer(ct_feat, center_pos_embedding)
    
with open(pickle_dir + "poolformer_output.pkl", 'rb') as handle:
    output_poolformer = pickle.load(handle)
    
print((output_poolformer[0][0][:50] == output[0][0][:50]).all())

tensor(True, device='cuda:0')


In [9]:
if POOLFORMER_GEN_ONNX:
    model_name = "poolformer.onnx"
    torch.onnx.export(poolformer, (ct_feat, center_pos_embedding), model_name, do_constant_folding=True,
                export_params=True, opset_version=11)


    !polygraphy surgeon sanitize poolformer.onnx --fold-constants --output poolformer_folded.onnx

    onnx.checker.check_model(onnx.load("poolformer_folded.onnx"))
    print("gen poolformer.onnx success!")
else:
    print("pass")

pass


In [10]:
# import torchvision.models as models

# resnet50_gpu = models.resnet50(pretrained=True, progress=False).to("cuda")
# resnet50_gpu